# PixelBytes: Catching Insights in Unified Multimodal Sequences

Ce notebook présente **PixelBytes**, un modèle innovant conçu pour générer simultanément du texte et des images pixel par pixel sous forme de séquences. L'objectif est d'explorer un embedding unifié qui permet une génération multimodale cohérente.

## Contexte et Architecture Proposée

### Fondements Théoriques
- **Image Transformer** : [Génération d'images pixel par pixel](https://arxiv.org/abs/1802.05751)
- **Bi-Mamba+** : [Modèle bidirectionnel pour la prévision de séries temporelles](https://arxiv.org/abs/2404.15772)
- **MambaByte** : [Modèle d'état d'espace sélectif sans token](https://arxiv.org/abs/2401.13660)

### Concept Clé
Le modèle PixelByte génère des séquences mixtes de texte et d'images. Il doit :
- Gérer les transitions entre texte et image avec des sauts de ligne (ASCII 0A).
- Maintenir la cohérence des dimensions des images générées.
- Assimiler la tâche de "copie" pour reproduire des motifs complexes.

Ce notebook utilise la puissance des GPU T4 x2 de Kaggle pour expérimenter avec des architectures avancées et des jeux de données volumineux, afin de relever les défis de la génération multimodale unifiée.

## Ressources du Projet

### Dataset
Pour ce projet, nous utiliserons le dataset **PixelBytes-Pokemon**, spécialement conçu pour cette tâche de génération multimodale. Ce dataset, créé par l'auteur de ce notebook, est disponible sur Hugging Face : [PixelBytes-Pokemon](https://huggingface.co/datasets/ffurfaro/PixelBytes-Pokemon). Il contient des séquences de texte et d'images de Pokémon, encodées de manière à permettre l'entraînement de notre modèle PixelByte sur des données multimodales.

### Implémentation
L'implémentation du modèle et les scripts d'entraînement sont disponibles dans le dépôt GitHub **Mamba-Bys** : [Mamba-Bys](https://github.com/fabienfrfr/Mamba-Bys). Ce dépôt contient le code source nécessaire pour reproduire les expériences, ainsi que des instructions détaillées sur la configuration et l'utilisation du modèle PixelByte.

## Modele à entrainer :

- 8 LSTM (bidirectionnel + 1,2,3 layers) + (p_embed + bi-2 layers)
- 6 Mamba (bidirectionnel + 1,2,3 layers)
- 3 Transformers (1,2,3 layers)

# Pre-test

Avant d'entrainer les 8 LSTM, prendre le pembed-bi-2 LSTM et tester la génération (à 40%, mais influence des répétitions de caractére et de pixel)

Forte chance de predire le meme pixel à la suite, et de prédire des espaces et voyelles.

Pour la generation, le modele génére uniquement le prochain element central. L'algorithme de génération doit reconstituer la structure 2D. Tel que : https://www.perplexity.ai/search/tu-peux-me-dire-combien-obtien-z6lt6R6IRrmwnILyCqwZxw

(T,T,\n,T,\n,P,P,P,\t,P,P,P,\n,T,T) donne 
([[0,0,0],[0,T,0],[0,0,0]],
[[0,0,0],[T,T,0],[0,0,0]],
[[0,0,0],[T,\n,0],[0,0,0]],
[[0,0,0],[\n,T,0],[0,0,0]],
[[0,0,0],[T,\n,0],[0,0,0]],
[[0,0,0],[0,P,0],[0,0,0]],
[[0,0,0],[P,P,0],[0,0,0]],
[[0,0,0],[P,P,0],[0,0,0]],
[[0,0,0],[P,\t,0],[0,0,0]],
[[0,P,P],[0,P,0],[0,0,0]],
[[P,P,P],[P,P,0],[0,0,0]],
[[P,P,P],[P,P,0],[0,0,0]],
[[P,\t,0],[P,\n,0],[0,0,0]],
[[0,0,0],[0,T,0],[0,0,0]],
[[0,0,0],[T,T,0],[0,0,0]],) avec P les pixel, \n les saut de ligne et de modalité, et \t, les changement de ligne de pixex de l'image et/ou les tabulations de texte.

In [14]:
!pip install -q git+https://github.com/fabienfrfr/PixelBytes.git@main

In [15]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from pixelbytes import *

In [16]:
# init
hf_dataset = load_dataset("ffurfaro/PixelBytes-Pokemon")
ds = hf_dataset["train"].train_test_split(test_size=0.1)

train_dataset = PxByDataset(ds["train"]["pixelbyte"], seq_length=256, stride=32)
test_dataset = PxByDataset(ds["test"]["pixelbyte"], seq_length=256, stride=32)

pixelbyte = PixelBytesTokenizer()

In [17]:
pixelbyte = PixelBytesTokenizer()
vocab_size = len(pixelbyte.vocab); print(vocab_size)
embedding_dim = 81
hidden_dim = 64
n_layer = 2
model_config = ModelConfig(dim=embedding_dim, d_state=hidden_dim, depth=n_layer, vocab_size=vocab_size)

113


In [18]:
model = SimpleRNNModel(config)
#model = SimpleTransformerModel(config)

In [19]:
train_config = TrainConfig(model=model, model_config=model_config, dataset_name="PixelBytes-Pokemon", hf_token=hf_token,
                           train_dataset=train_dataset,test_dataset=test_dataset, num_epochs=200, repo_name="PixelBytes-Pokemon")
trainer = Trainer(train_config)

Complete path of pytorch model '.pth': models/rnn_bi_pxby_81-dim_64-state_2-layer_PixelBytes-Pokemon


In [20]:
trainer.train_and_evaluate()

Training:   2%|▎         | 5/200 [00:35<23:12,  7.14s/it]


Epoch 5: Train Loss: 1.9144, Test Loss: 1.9729, Test Acc: 45.85%


Training:   5%|▌         | 10/200 [01:10<22:25,  7.08s/it]


Epoch 10: Train Loss: 1.5816, Test Loss: 1.7508, Test Acc: 52.28%


Training:   8%|▊         | 15/200 [01:45<21:46,  7.06s/it]

...


Epoch 200: Train Loss: 0.2790, Test Loss: 2.7956, Test Acc: 52.28%
Repository 'ffurfaro/PixelBytes-Pokemon' created or already exists.


model.safetensors:   0%|          | 0.00/316k [00:00<?, ?B/s]

Model pushed successfully to PixelBytes-Pokemon, subfolder: rnn_bi_pxby_81_dim_64_state_2_layer_best
Repository 'ffurfaro/PixelBytes-Pokemon' created or already exists.


model.safetensors:   0%|          | 0.00/316k [00:00<?, ?B/s]

Model pushed successfully to PixelBytes-Pokemon, subfolder: rnn_bi_pxby_81_dim_64_state_2_layer_last
Training completed. Results and models saved.
